In [ ]:
import mlflow
import numpy as np

from utils.dataImportation import *

from utils.evaluation import *
from utils.performance_tables import *
from utils.staticModels import SplitDeepFactorNNList

In [ ]:
path_to_database = '../../data'
STOCK = 'SPX'
DIVFM_MODEL_RUN_ID = '7abae1944cc24028a5f66fb765614e96'
BENCHMARK_MODEL_RUN_ID = 'a060e99c3c1e4690974acbdf971d8922'
USE_MLFLOW = True

In [ ]:
DIVFM_model: SplitDeepFactorNNList = mlflow.pytorch.load_model(f"runs:/{DIVFM_MODEL_RUN_ID}/model")
assert isinstance(DIVFM_model, SplitDeepFactorNNList), f"Loaded model is {type(DIVFM_model)}"
benchmark_model: SplitDeepFactorNNList = mlflow.pytorch.load_model(f"runs:/{BENCHMARK_MODEL_RUN_ID}/model")
assert isinstance(benchmark_model, SplitDeepFactorNNList), f"Loaded model is {type(benchmark_model)}"

In [ ]:
FEATURES_NAME = ['ttm', 'ttm_scaled_moneyness'] if STOCK == 'SPX' else ['ttm', 'ttm_scaled_moneyness', 'time_to_earnings']
USE_LOG_VOL = True
BEGIN_TRAIN_DATE = '2000-01-01'
END_TRAIN_DATE = '2021-01-01'
BEGIN_TEST_DATE = '2022-01-01'

client = mlflow.MlflowClient()
client = mlflow.tracking.MlflowClient()
run_id = DIVFM_MODEL_RUN_ID

In [ ]:
labels_name = ['LOG_OM_IV'] if USE_LOG_VOL else ['OM_IV']
model_output_fn = torch.exp if USE_LOG_VOL else torch.nn.Identity()


In [ ]:
options_data = import_data(path_to_database, STOCK)
earning_dates = import_earnings_date(path_to_database, STOCK)
options_data = merge_options_earnings(options_data, earning_dates)
options_data = apply_filters_to_options(options_data)
if STOCK != 'SPX': options_data = generate_tte(options_data, earning_dates)

train_data, valid_data, test_data = split_in_sets(options_data.dropna(), BEGIN_TRAIN_DATE, END_TRAIN_DATE, BEGIN_TEST_DATE)
train_dataset, valid_dataset, test_dataset = get_datasets(train_data, valid_data, test_data,
                                                          FEATURES_NAME, labels_name, dtype=torch.float32)

In [ ]:
DIVFM_full_data, DIVFM_daily_data = create_full_and_daily_datasets(DIVFM_model, train_dataset,
                                                                 valid_dataset, test_dataset, earning_dates)
benchmark_full_data, benchmark_daily_data = create_full_and_daily_datasets(benchmark_model, train_dataset,
                                                                       valid_dataset, test_dataset, earning_dates)

In [ ]:
M_BINS = (-np.inf, -0.5, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.5, np.inf)
TAU_BINS = (0, 5, 35, 75, 110, 145, 220, 290, 400, 620, np.inf)


DIVFM_full_data = add_tau_moneyness_bins(DIVFM_full_data, m_bins=M_BINS, tau_bins=TAU_BINS)
benchmark_full_data = add_tau_moneyness_bins(benchmark_full_data, m_bins=M_BINS, tau_bins=TAU_BINS)

In [ ]:
DIVFM_LABEL = "RMSE - DIVFM"
BENCHMARK_LABEL = "RMSE - BENCHMARK"
M_LABELS_ORDER = tuple(f"M{i+1}" for i in range(10))
M_LABELS_ORDER_MATH = tuple(rf"$\text{{M}}_{i+1}$" for i in range(10)) + ('All',)
TAU_LABELS_ORDER = tuple(f"tau{i+1}" for i in range(10))
TAU_LABELS_ORDER_MATH = tuple(rf"$\\tau_{i+1}$" for i in range(10)) + ("All",)

In [ ]:
DIVFM_m_summary = summarize_by_moneyness_or_tau_full(DIVFM_full_data,
                                        bin_col = "moneyness_bin",
                                        moneyness_or_tau_col = "ttm_scaled_moneyness",
                                        labels_order=M_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = None,
                                        label1 = DIVFM_LABEL,
                                        label2 = None,
                                        split_data=False
                                        )
DIVFM_tau_summary = summarize_by_moneyness_or_tau_full(DIVFM_full_data,
                                        bin_col = "tau_bin",
                                        moneyness_or_tau_col = "days_to_maturity",
                                        labels_order=TAU_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = None,
                                        label1 = DIVFM_LABEL,
                                        label2 = None, 
                                        split_data=False,
                                        )

benchmark_m_summary = summarize_by_moneyness_or_tau_full(benchmark_full_data,
                                        bin_col = "moneyness_bin",
                                        moneyness_or_tau_col = "ttm_scaled_moneyness",
                                        labels_order=M_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = None,
                                        label1 = BENCHMARK_LABEL,
                                        label2 = None, 
                                        split_data=False
                                        )
benchmark_tau_summary = summarize_by_moneyness_or_tau_full(benchmark_full_data,
                                        bin_col = "tau_bin",
                                        moneyness_or_tau_col = "days_to_maturity",
                                        labels_order=TAU_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = None,
                                        label1 = BENCHMARK_LABEL,
                                        label2 = None, 
                                        split_data=False
                                        )

In [ ]:
fig = plot_rmse_by_bins_two_panel(
    m_summary_a=benchmark_m_summary,
    m_summary_b=DIVFM_m_summary,
    m_bins=M_BINS,
    m_symbol="M",

    tau_summary_a=benchmark_tau_summary,
    tau_summary_b=DIVFM_tau_summary,
    tau_bins=TAU_BINS,
    tau_symbol="TTM",

    label_a="Benchmark",
    label_b="DIVFM",
)
client.log_figure(run_id=run_id, figure=fig, artifact_file=f'RMSE_comparison_Benchmark_vs_DIVFM.png')
plt.show()

In [ ]:
fig, axs = plot_daily_rmse_timeseries_with_diff(
    ttea_daily_data=DIVFM_daily_data,
    no_ttea_daily_data=benchmark_daily_data,
    label_a_suffix='DIVFM',
    label_b_suffix='Benchmark',
)
client.log_figure(run_id=run_id, figure=fig, artifact_file=f'RMSE_timeseries_comparison_benchmark_DIVFM.png')

In [ ]:
LABEL1 = "RMSE - DIVFM"
LABEL2 = "RMSE - Benchmark"
M_LABELS_ORDER = tuple(f"M{i+1}" for i in range(10))
M_LABELS_ORDER_MATH = tuple(rf"$\text{{M}}_{{{i+1}}}$" for i in range(10)) + ('All',)
TAU_LABELS_ORDER = tuple(f"tau{i+1}" for i in range(10))
TAU_LABELS_ORDER_MATH = tuple(rf"$\tau_{{{i+1}}}$" for i in range(10)) + ("All",)

In [ ]:
full_data = DIVFM_full_data
full_data['IVpred_benchmark'] = benchmark_full_data['IVpred']

m_summary = summarize_by_moneyness_or_tau_full(full_data,
                                        bin_col = "moneyness_bin",
                                        moneyness_or_tau_col = "ttm_scaled_moneyness",
                                        labels_order=M_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = "IVpred_benchmark",
                                        label1 = LABEL1,
                                        label2 = LABEL2, 
                                        )
tau_summary = summarize_by_moneyness_or_tau_full(full_data,
                                        bin_col = "tau_bin",
                                        moneyness_or_tau_col = "days_to_maturity",
                                        labels_order=TAU_LABELS_ORDER,
                                        pred_col1 = "IVpred",
                                        pred_col2 = "IVpred_benchmark",
                                        label1 = LABEL1,
                                        label2 = LABEL2, 
                                        )

moneyness_str = bins_table_to_latex(m_summary, label1=LABEL1, label2=LABEL2, bin_headers_math=M_LABELS_ORDER_MATH)
tau_str = bins_table_to_latex(tau_summary, label1=LABEL1, label2=LABEL2, bin_headers_math=TAU_LABELS_ORDER_MATH)


client.log_table(run_id=run_id, data=summary_to_long_df(m_summary),
                 artifact_file=f"performance_tables_comparison/moneyness_table.json")
client.log_table(run_id=run_id, data=summary_to_long_df(tau_summary),
                 artifact_file=f"performance_tables_comparison/ttm_table.json")
client.log_text(run_id=run_id, text=moneyness_str,
                artifact_file="performance_tables_comparison/moneyness_latex_table.txt")
client.log_text(run_id=run_id, text=tau_str,
                artifact_file="performance_tables_comparison/ttm_latex_table.txt")